In [2]:
import os
import shutil
from datetime import datetime
import sqlite3
import logging

def move_processed_files(processing_folder="processing/", completed_folder="completed/"):
    try:
        # Get current date
        today_date = datetime.now().strftime("%y%m%d")

        # Get list of files in today's date folder in the processing directory
        processing_folder_today = os.path.join(processing_folder, today_date)
        if not os.path.exists(processing_folder_today):
            logging.info(f"No files found for today's date {today_date} in the processing folder.")
            return
        
        files = os.listdir(processing_folder_today)

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Iterate through each file
        for file in files:
            # Get the full path of the file
            file_path = os.path.join(processing_folder_today, file)

            # Move the file to the completed folder
            completed_folder_today = os.path.join(completed_folder, today_date)
            os.makedirs(completed_folder_today, exist_ok=True)
            shutil.move(file_path, completed_folder_today)

            # Update status in the ProcessedFile table
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute('''UPDATE ProcessedFile 
                              SET status=?, updated_datetime=? 
                              WHERE local_file=?''', 
                           ('Completed', now, file))

            # Update updated_datetime in SourceFile table
            cursor.execute('''UPDATE SourceFile 
                              SET updated_datetime=? 
                              WHERE source_file=?''', 
                           (now, file))

        # Commit changes to the database
        conn.commit()
        conn.close()
        
    except Exception as e:
        logging.error(f"Error moving processed files: {str(e)}")

if __name__ == "__main__":
    move_processed_files()


In [1]:
import os
import shutil
import sqlite3
from datetime import datetime

def move_files_to_completed():
    try:
        # Constants
        processing_folder = "processing/"
        completed_folder = "completed/"

        # Get current date
        today_date = datetime.now().strftime("%y%m%d")

        # Path to today's processing folder
        processing_folder_today = os.path.join(processing_folder, today_date)
        if not os.path.exists(processing_folder_today):
            print(f"No files found for today's date {today_date} in the processing folder.")
            return
        
        # Path to today's completed folder
        completed_folder_today = os.path.join(completed_folder, today_date)
        os.makedirs(completed_folder_today, exist_ok=True)

        # Get list of files in today's processing folder
        files = os.listdir(processing_folder_today)

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Iterate through each file
        for file_name in files:
            # Move file to completed folder
            source_path = os.path.join(processing_folder_today, file_name)
            destination_path = os.path.join(completed_folder_today, file_name)
            shutil.move(source_path, destination_path)

            # Update ProcessedFile table
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute('''UPDATE ProcessedFile 
                              SET status=?, updated_datetime=? 
                              WHERE local_file=?''', 
                           ('Completed', now, file_name))
            conn.commit()

            # Update SourceFile table
            cursor.execute('''UPDATE SourceFile 
                              SET status=?, updated_datetime=? 
                              WHERE source_file=?''', 
                           ('Completed', now, file_name))
            conn.commit()

        # Close the database connection
        conn.close()
    except Exception as e:
        print(f"Error moving files to completed folder: {str(e)}")

# Call the function to move files to completed folder and update database
move_files_to_completed()


In [6]:
import os
import shutil
from datetime import datetime
import subprocess
import sqlite3
import logging

# Set up logging
log_file = 'logs/file_convert.log'
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

input_folder = "input/"
processing_folder = "processing/"
completed_folder = "completed/"

def move_to_completed_folder():
    try:
        # Get current date
        today_date = datetime.now().strftime("%y%m%d")
        
        # Source directory path
        source_dir = os.path.join(processing_folder, today_date)
        
        # Destination directory path
        dest_dir = os.path.join(completed_folder, today_date)
        
        # Move files from processing folder to completed folder
        if os.path.exists(source_dir):
            os.makedirs(dest_dir, exist_ok=True)
            for file in os.listdir(source_dir):
                shutil.move(os.path.join(source_dir, file), os.path.join(dest_dir, file))
                print(f"Moved file: {file} from {source_dir} to {dest_dir}")  # Added print statement
            logging.info(f"Files moved from processing folder to completed folder for date: {today_date}")
    except Exception as e:
        logging.error(f"Error moving files to completed folder: {str(e)}")

def update_database():
    try:
        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()
        
        # Get current date
        today_date = datetime.now().strftime("%y%m%d")
        
        # Update ProcessedFile table
        cursor.execute('''UPDATE ProcessedFile 
                          SET status=?, updated_datetime=? 
                          WHERE status=? AND chunk_created_datetime LIKE ?''', 
                       ('Completed', datetime.now().strftime("%Y-%m-%d %H:%M:%S"), 'processing', f'{today_date}%'))
        print("ProcessedFile table update SQL:", cursor.statement)  # Added print statement
        
        # Update SourceFile table
        cursor.execute('''UPDATE SourceFile 
                          SET status=?, updated_datetime=? 
                          WHERE status=? AND updated_datetime LIKE ?''', 
                       ('Completed', datetime.now().strftime("%Y-%m-%d %H:%M:%S"), 'processing', f'{today_date}%'))
        print("SourceFile table update SQL:", cursor.statement)  # Added print statement
        
        # Commit changes and close connection
        conn.commit()
        conn.close()
        logging.info("Database updated successfully.")
    except Exception as e:
        logging.error(f"Error updating database: {str(e)}")

def create_chunks(input_file, output_folder, chunk_duration=10):
    try:
        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Create table if not exists
        cursor.execute('''CREATE TABLE IF NOT EXISTS ProcessedFile 
                          (id INTEGER PRIMARY KEY AUTOINCREMENT, 
                          local_file TEXT, 
                          source_file TEXT, 
                          status TEXT, 
                          chunk_created_datetime TEXT,
                          updated_datetime TEXT)''')
        conn.commit()

        # Update database with processing status and start datetime
        start_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        # Extract the filename without the path
        file_name = os.path.basename(input_file)
        cursor.execute('''SELECT * FROM SourceFile WHERE source_file=?''', (file_name,))
        row = cursor.fetchone()
        if row:
            # Entry exists, update it
            cursor.execute('''UPDATE SourceFile 
                              SET status=?, updated_datetime=? 
                              WHERE source_file=?''', 
                           ('processing', start_datetime, file_name))
        else:
            # Entry doesn't exist, create it
            cursor.execute('''INSERT INTO SourceFile (source_file, status, updated_datetime) 
                              VALUES (?, ?, ?)''', (file_name, 'processing', start_datetime))
        conn.commit()

        # Create folder for the input file
        today_date = datetime.now().strftime("%y%m%d")
        file_name_no_extension = os.path.splitext(file_name)[0]
        file_folder = os.path.join(output_folder, today_date, file_name_no_extension)
        os.makedirs(file_folder, exist_ok=True)

        # Convert folder
        convert_folder = os.path.join(file_folder, "Convert")
        os.makedirs(convert_folder, exist_ok=True)

        # Chunks folder
        chunks_folder = os.path.join(file_folder, "Chunks")
        os.makedirs(chunks_folder, exist_ok=True)

        # Check if WMA file already exists
        output_file_wma = os.path.join(convert_folder, f"{file_name_no_extension}.wma")
        if os.path.exists(output_file_wma):
            logging.info(f"Skipping conversion for '{file_name}', WMA file already exists.")
            # Update status of the source file to 'processed'
            end_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute('''UPDATE SourceFile 
                              SET status=?, updated_datetime=? 
                              WHERE source_file=?''', 
                           ('Completed', end_datetime, file_name))
            conn.commit()
            conn.close()
            return

        # Convert WAV to WMA using FFmpeg
        logging.info(f"Converting '{file_name}' to WMA format.")
        subprocess.run(['ffmpeg', '-y', '-i', input_file, '-acodec', 'wmav2', output_file_wma], check=True)

        # Extract audio duration
        duration = subprocess.check_output(['ffprobe', '-i', output_file_wma, '-show_entries', 'format=duration', '-v', 'quiet', '-of', 'csv=%s' % ("p=0")])
        duration = float(duration.strip())

        # Calculate number of chunks
        total_chunks = int(duration / chunk_duration)
        logging.info(f"Creating {total_chunks} chunks for '{file_name}'.")

        # Iterate through chunks
        for i in range(total_chunks):
            # Calculate start and end time for the chunk
            start_time = i * chunk_duration
            end_time = start_time + chunk_duration

            # Create filename for the chunk
            chunk_name = f"{file_name_no_extension}_{start_time}_{end_time}.wma"
            output_file_chunk = os.path.join(chunks_folder, chunk_name)

            # Extract the chunk using FFmpeg
            logging.info(f"Extracting chunk {i + 1}/{total_chunks} for '{file_name}'.")
            subprocess.run(['ffmpeg', '-y', '-i', output_file_wma, '-acodec', 'copy', '-ss', str(start_time), '-to', str(end_time), output_file_chunk], check=True)

            # Update ProcessedFile table
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute('''INSERT INTO ProcessedFile 
                              (local_file, source_file, status, chunk_created_datetime, updated_datetime) 
                              VALUES (?, ?, ?, ?, ?)''', 
                           (os.path.basename(output_file_chunk), file_name, 'processing', now, now))
            conn.commit()

        # Update status of the source file to 'processed'
        end_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cursor.execute('''UPDATE SourceFile 
                          SET status=?, updated_datetime=? 
                          WHERE source_file=?''', 
                       ('Completed', end_datetime, file_name))
        conn.commit()

        # Close the database connection
        conn.close()
    except Exception as e:
        logging.error(f"Error processing file {input_file}: {str(e)}")

def process_audio_files():
    try:
        # Get current date
        today_date = datetime.now().strftime("%y%m%d")

        # Get list of files in today's date folder in the input directory
        input_folder_today = os.path.join(input_folder, today_date)
        if not os.path.exists(input_folder_today):
            logging.info(f"No files found for today's date {today_date} in the input folder.")
            return
        
        files = os.listdir(input_folder_today)

        # Iterate through each file
        for file in files:
            # Check if it's a WAV file
            if file.endswith(".wav"):
                # Create chunks from the audio file
                create_chunks(os.path.join(input_folder_today, file), processing_folder)
        
        # Move processed files to completed folder
        move_to_completed_folder()
        
        # Update database
        update_database()
    except Exception as e:
        logging.error(f"Error processing audio files: {str(e)}")

# Call the function to process audio files
process_audio_files()


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Moved file: 02C36052-9A9B-EB11-9D8F-D89D67F437E7 from processing/240418 to completed/240418
Moved file: 7B54F528-9F9B-EB11-9D8F-D89D67F437E7 from processing/240418 to completed/240418
Moved file: 3E4A40A4-9B9B-EB11-9DAE-E8393530AAAE from processing/240418 to completed/240418
Moved file: 5A24DAA3-999B-EB11-9D8F-D89D67F437E7 from processing/240418 to completed/240418
Moved file: 1CAFA997-9E9B-EB11-9D8F-D89D67F437E7 from processing/240418 to completed/240418


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [7]:
import os
import shutil
from datetime import datetime
import subprocess
import sqlite3
import logging

# Set up logging
log_file = 'logs/file_convert.log'
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

input_folder = "input/"
processing_folder = "processing/"

def create_chunks(input_file, output_folder, chunk_duration=10):
    try:
        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Create table if not exists
        cursor.execute('''CREATE TABLE IF NOT EXISTS ProcessedFile 
                          (id INTEGER PRIMARY KEY AUTOINCREMENT, 
                          local_file TEXT, 
                          source_file TEXT, 
                          status TEXT, 
                          chunk_created_datetime TEXT,
                          updated_datetime TEXT)''')
        conn.commit()

        # Update database with processing status and start datetime
        start_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        # Extract the filename without the path
        file_name = os.path.basename(input_file)
        cursor.execute('''SELECT * FROM SourceFile WHERE source_file=?''', (file_name,))
        row = cursor.fetchone()
        if row:
            # Entry exists, update it
            cursor.execute('''UPDATE SourceFile 
                              SET status=?, updated_datetime=? 
                              WHERE source_file=?''', 
                           ('processing', start_datetime, file_name))
        else:
            # Entry doesn't exist, create it
            cursor.execute('''INSERT INTO SourceFile (source_file, status, updated_datetime) 
                              VALUES (?, ?, ?)''', (file_name, 'processing', start_datetime))
        conn.commit()

        # Create folder for the input file
        today_date = datetime.now().strftime("%y%m%d")
        file_name_no_extension = os.path.splitext(file_name)[0]
        file_folder = os.path.join(output_folder, today_date, file_name_no_extension)
        os.makedirs(file_folder, exist_ok=True)

        # Convert folder
        convert_folder = os.path.join(file_folder, "Convert")
        os.makedirs(convert_folder, exist_ok=True)

        # Chunks folder
        chunks_folder = os.path.join(file_folder, "Chunks")
        os.makedirs(chunks_folder, exist_ok=True)

        # Check if WMA file already exists
        output_file_wma = os.path.join(convert_folder, f"{file_name_no_extension}.wma")
        if os.path.exists(output_file_wma):
            logging.info(f"Skipping conversion for '{file_name}', WMA file already exists.")
            # Update status of the source file to 'processed'
            end_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute('''UPDATE SourceFile 
                              SET status=?, updated_datetime=? 
                              WHERE source_file=?''', 
                           ('Completed', end_datetime, file_name))
            conn.commit()
            conn.close()
            return

        # Convert WAV to WMA using FFmpeg
        logging.info(f"Converting '{file_name}' to WMA format.")
        subprocess.run(['ffmpeg', '-y', '-i', input_file, '-acodec', 'wmav2', output_file_wma], check=True)

        # Extract audio duration
        duration = subprocess.check_output(['ffprobe', '-i', output_file_wma, '-show_entries', 'format=duration', '-v', 'quiet', '-of', 'csv=%s' % ("p=0")])
        duration = float(duration.strip())

        # Calculate number of chunks
        total_chunks = int(duration / chunk_duration)
        logging.info(f"Creating {total_chunks} chunks for '{file_name}'.")

        # Iterate through chunks
        for i in range(total_chunks):
            # Calculate start and end time for the chunk
            start_time = i * chunk_duration
            end_time = start_time + chunk_duration

            # Create filename for the chunk
            chunk_name = f"{file_name_no_extension}_{start_time}_{end_time}.wma"
            output_file_chunk = os.path.join(chunks_folder, chunk_name)

            # Extract the chunk using FFmpeg
            logging.info(f"Extracting chunk {i + 1}/{total_chunks} for '{file_name}'.")
            subprocess.run(['ffmpeg', '-y', '-i', output_file_wma, '-acodec', 'copy', '-ss', str(start_time), '-to', str(end_time), output_file_chunk], check=True)

            # Update ProcessedFile table
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute('''INSERT INTO ProcessedFile 
                              (local_file, source_file, status, chunk_created_datetime, updated_datetime) 
                              VALUES (?, ?, ?, ?, ?)''', 
                           (os.path.basename(output_file_chunk), file_name, 'processing', now, now))
            conn.commit()

        # Update status of the source file to 'completed' after all chunks are processed
        end_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cursor.execute('''UPDATE SourceFile 
                          SET status=?, updated_datetime=? 
                          WHERE source_file=?''', 
                       ('Completed', end_datetime, file_name))
        conn.commit()

        # Close the database connection
        conn.close()
    except Exception as e:
        logging.error(f"Error processing file {input_file}: {str(e)}")

def process_audio_files():
    try:
        # Get current date
        today_date = datetime.now().strftime("%y%m%d")

        # Get list of files in today's date folder in the input directory
        input_folder_today = os.path.join(input_folder, today_date)
        if not os.path.exists(input_folder_today):
            logging.info(f"No files found for today's date {today_date} in the input folder.")
            return
        
        files = os.listdir(input_folder_today)

        # Iterate through each file
        for file in files:
            # Check if it's a WAV file
            if file.endswith(".wav"):
                # Create chunks from the audio file
                create_chunks(os.path.join(input_folder_today, file), processing_folder)
    except Exception as e:
        logging.error(f"Error processing audio files: {str(e)}")

# Call the function to process audio files
process_audio_files()


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [3]:
import os
import shutil
import sqlite3
from datetime import datetime

def move_files_to_completed():
    try:
        # Constants
        processing_folder = "processing/"
        completed_folder = "completed/"

        # Get current date
        today_date = datetime.now().strftime("%y%m%d")

        # Path to today's processing folder
        processing_folder_today = os.path.join(processing_folder, today_date)
        if not os.path.exists(processing_folder_today):
            print(f"No files found for today's date {today_date} in the processing folder.")
            return
        
        # Path to today's completed folder
        completed_folder_today = os.path.join(completed_folder, today_date)
        os.makedirs(completed_folder_today, exist_ok=True)

        # Get list of files in today's processing folder
        files = os.listdir(processing_folder_today)

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Iterate through each file
        for file_name in files:
            # Move file to completed folder
            source_path = os.path.join(processing_folder_today, file_name)
            destination_path = os.path.join(completed_folder_today, file_name)
            shutil.move(source_path, destination_path)

            # Update ProcessedFile table
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute('''UPDATE ProcessedFile 
                              SET status=?, updated_datetime=? 
                              WHERE local_file=?''', 
                           ('Completed', now, file_name))
            conn.commit()

            # Print debug information
            print(f"Updated ProcessedFile table for file: {file_name}")

            # Update SourceFile table
            cursor.execute('''UPDATE SourceFile 
                              SET status=?, updated_datetime=? 
                              WHERE source_file=?''', 
                           ('Completed', now, file_name))
            conn.commit()

            # Print debug information
            print(f"Updated SourceFile table for file: {file_name}")

        # Close the database connection
        conn.close()
    except Exception as e:
        print(f"Error moving files to completed folder: {str(e)}")

# Call the function to move files to completed folder and update database
move_files_to_completed()


Updated ProcessedFile table for file: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE
Updated SourceFile table for file: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE
Updated ProcessedFile table for file: 1247DB66-9C9B-EB11-9D8F-D89D67F437E7
Updated SourceFile table for file: 1247DB66-9C9B-EB11-9D8F-D89D67F437E7
Updated ProcessedFile table for file: EC65ABC6-9F9B-EB11-B770-E8393530AA84
Updated SourceFile table for file: EC65ABC6-9F9B-EB11-B770-E8393530AA84
Updated ProcessedFile table for file: 054A40A4-9B9B-EB11-9DAE-E8393530AAAE
Updated SourceFile table for file: 054A40A4-9B9B-EB11-9DAE-E8393530AAAE
Updated ProcessedFile table for file: D046DB66-9C9B-EB11-9D8F-D89D67F437E7
Updated SourceFile table for file: D046DB66-9C9B-EB11-9D8F-D89D67F437E7


In [4]:
import os
import shutil
import sqlite3
from datetime import datetime

def move_files_to_completed():
    try:
        # Constants
        processing_folder = "processing/"
        completed_folder = "completed/"

        # Get current date
        today_date = datetime.now().strftime("%y%m%d")

        # Path to today's processing folder
        processing_folder_today = os.path.join(processing_folder, today_date)
        if not os.path.exists(processing_folder_today):
            print(f"No files found for today's date {today_date} in the processing folder.")
            return
        
        # Path to today's completed folder
        completed_folder_today = os.path.join(completed_folder, today_date)
        os.makedirs(completed_folder_today, exist_ok=True)

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Get list of files in today's processing folder
        files = os.listdir(processing_folder_today)

        # Iterate through each file
        for file_name in files:
            # Path to chunks folder for the current file
            file_folder = os.path.join(processing_folder_today, file_name)
            chunks_folder = os.path.join(file_folder, "Chunks")
            
            # Check if chunks folder exists
            if os.path.exists(chunks_folder):
                # Get list of files in the chunks folder
                chunks_files = os.listdir(chunks_folder)
                
                # Iterate through each chunk file
                for chunk_file in chunks_files:
                    # Move chunk file to completed folder
                    source_path = os.path.join(chunks_folder, chunk_file)
                    destination_path = os.path.join(completed_folder_today, chunk_file)
                    shutil.move(source_path, destination_path)

                    # Update ProcessedFile table
                    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    cursor.execute('''INSERT INTO ProcessedFile 
                                      (local_file, source_file, status, chunk_created_datetime, updated_datetime) 
                                      VALUES (?, ?, ?, ?, ?)''', 
                                   (chunk_file, file_name, 'Completed', now, now))
                    conn.commit()

                    # Print debug information
                    print(f"Updated ProcessedFile table for chunk file: {chunk_file} in folder: {file_name}")

            # Move main file to completed folder
            source_path = os.path.join(processing_folder_today, file_name)
            destination_path = os.path.join(completed_folder_today, file_name)
            shutil.move(source_path, destination_path)

            # Update ProcessedFile table for main file
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute('''UPDATE ProcessedFile 
                              SET status=?, updated_datetime=? 
                              WHERE local_file=?''', 
                           ('Completed', now, file_name))
            conn.commit()

            # Print debug information
            print(f"Updated ProcessedFile table for main file: {file_name}")

            # Update SourceFile table
            cursor.execute('''UPDATE SourceFile 
                              SET status=?, updated_datetime=? 
                              WHERE source_file=?''', 
                           ('Completed', now, file_name))
            conn.commit()

            # Print debug information
            print(f"Updated SourceFile table for file: {file_name}")

        # Close the database connection
        conn.close()
    except Exception as e:
        print(f"Error moving files to completed folder: {str(e)}")

# Call the function to move files to completed folder and update database
move_files_to_completed()


Updated ProcessedFile table for chunk file: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_20_30.wma in folder: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE
Updated ProcessedFile table for chunk file: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_40_50.wma in folder: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE
Updated ProcessedFile table for chunk file: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_0_10.wma in folder: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE
Updated ProcessedFile table for chunk file: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_10_20.wma in folder: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE
Updated ProcessedFile table for chunk file: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_30_40.wma in folder: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE
Updated ProcessedFile table for main file: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE
Updated SourceFile table for file: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE
Updated ProcessedFile table for chunk file: 1247DB66-9C9B-EB11-9D8F-D89D67F437E7_40_50.wma in folder: 1247DB66-9C9B-EB11-9D8F-D89D67F437E7
Updated Process

In [23]:
import os
import shutil
import sqlite3
from datetime import datetime

def move_files_to_completed():
    try:
        # Constants
        processing_folder = "processing/"
        completed_folder = "completed/"

        # Get current date
        today_date = datetime.now().strftime("%y%m%d")

        # Path to today's processing folder
        processing_folder_today = os.path.join(processing_folder, today_date)
        if not os.path.exists(processing_folder_today):
            print(f"No files found for today's date {today_date} in the processing folder.")
            return
        
        # Path to today's completed folder
        completed_folder_today = os.path.join(completed_folder, today_date)
        os.makedirs(completed_folder_today, exist_ok=True)

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Get list of files in today's processing folder
        files = os.listdir(processing_folder_today)

        # Iterate through each file
        for file_name in files:
            # Move file to completed folder
            source_path = os.path.join(processing_folder_today, file_name)
            destination_path = os.path.join(completed_folder_today, file_name)
            shutil.move(source_path, destination_path)

            # Update ProcessedFile table for main file
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute('''UPDATE ProcessedFile 
                              SET status=?, updated_datetime=? 
                              WHERE local_file=?''', 
                           ('Completed', now, file_name))
            conn.commit()

            # Update SourceFile table
            cursor.execute('''UPDATE SourceFile 
                              SET status=?, updated_datetime=? 
                              WHERE source_file=?''', 
                           ('Completed', now, file_name))
            conn.commit()

            # Path to chunks folder for the current file
            file_folder = os.path.join(completed_folder_today, file_name)
            chunks_folder = os.path.join(file_folder, "Chunks")
            
            # Check if chunks folder exists
            if os.path.exists(chunks_folder):
                # Get list of files in the chunks folder
                chunks_files = os.listdir(chunks_folder)
                
                # Iterate through each chunk file
                for chunk_file in chunks_files:
                    # Update ProcessedFile table for chunk file
                    cursor.execute('''INSERT INTO ProcessedFile 
                                      (local_file, source_file, status, chunk_created_datetime, updated_datetime) 
                                      VALUES (?, ?, ?, ?, ?)''', 
                                   (chunk_file, file_name, 'Completed', now, now))
                    conn.commit()

        # Close the database connection
        conn.close()
    except Exception as e:
        print(f"Error moving files to completed folder: {str(e)}")

# Call the function to move files to completed folder and update database
move_files_to_completed()


Error moving files to completed folder: Destination path 'completed/240419/8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE/8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE' already exists


In [15]:
import os
import shutil
import sqlite3
from datetime import datetime

def move_files_to_completed():
    try:
        # Constants
        processing_folder = "processing/"
        completed_folder = "completed/"

        # Get current date
        today_date = datetime.now().strftime("%y%m%d")

        # Path to today's processing folder
        processing_folder_today = os.path.join(processing_folder, today_date)
        if not os.path.exists(processing_folder_today):
            print(f"No files found for today's date {today_date} in the processing folder.")
            return
        
        # Path to today's completed folder
        completed_folder_today = os.path.join(completed_folder, today_date)
        os.makedirs(completed_folder_today, exist_ok=True)

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Get list of files in today's processing folder
        files = os.listdir(processing_folder_today)

        # Iterate through each file
        for file_name in files:
            print(f"Moving file {file_name} to completed folder...")

            # Move file to completed folder
            source_path = os.path.join(processing_folder_today, file_name)
            destination_path = os.path.join(completed_folder_today, file_name)
            shutil.move(source_path, destination_path)
            print(f"File {file_name} moved to completed folder.")

            # Update SourceFile table
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute('''UPDATE SourceFile 
                              SET status=?, updated_datetime=? 
                              WHERE source_file=?''', 
                           ('Ready', now, file_name + ".wav"))
            conn.commit()
            print(f"Updated SourceFile table for {file_name}.")

            # Path to chunks folder for the current file
            file_folder = os.path.join(completed_folder_today, file_name)
            print("Folder name:",file_folder)
            chunks_folder = os.path.join(file_folder, "Chunks")
            print("chunks name:",chunks_folder)
            
           # Check if chunks folder exists
            if os.path.exists(chunks_folder):
                print(f"Processing chunks for {file_name}...")
                # Get list of files in the chunks folder
                chunks_files = os.listdir(chunks_folder)
                
                # Iterate through each chunk file
                for chunk_file in chunks_files:
                    print("chunk file name:", chunk_file)
                    try:
                        # Update ProcessedFile table for chunk file
                        cursor.execute('''INSERT INTO ProcessedFile 
                                        (local_file, source_file, status, chunk_created_datetime, updated_datetime) 
                                        VALUES (?, ?, ?, ?, ?)''', 
                                    (chunk_file, file_name, 'Completed', now, now))
                        conn.commit()
                        print(f"Updated: {chunk_file}.")
                    except Exception as e:
                        print(f"Error updating ProcessedFile table for chunk {chunk_file}: {str(e)}")
            else:
                print("Not Found CHunks folder")

        # Close the database connection
        conn.close()
        print("All files processed successfully.")
    except Exception as e:
        print(f"Error moving files to completed folder: {str(e)}")

# Call the function to move files to completed folder and update database
move_files_to_completed()


Moving file 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE to completed folder...
File 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE moved to completed folder.
Updated SourceFile table for 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE.
Folder name: completed/240419/8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE
chunks name: completed/240419/8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE/Chunks
Processing chunks for 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE...
chunk file name: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_20_30.wma
Updated: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_20_30.wma.
chunk file name: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_40_50.wma
Updated: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_40_50.wma.
chunk file name: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_0_10.wma
Updated: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_0_10.wma.
chunk file name: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_10_20.wma
Updated: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_10_20.wma.
chunk file name: 8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE_30_40.wma
Updated: 8CA50AF3-9A9B-EB11-9DAE-E8393530AA

In [24]:
import os
import shutil
import sqlite3
from datetime import datetime

def move_files_to_completed():
    try:
        # Constants
        processing_folder = "processing/"
        completed_folder = "completed/"

        # Get current date
        today_date = datetime.now().strftime("%y%m%d")

        # Path to today's processing folder
        processing_folder_today = os.path.join(processing_folder, today_date)
        if not os.path.exists(processing_folder_today):
            print(f"No files found for today's date {today_date} in the processing folder.")
            return
        
        # Path to today's completed folder
        completed_folder_today = os.path.join(completed_folder, today_date)
        os.makedirs(completed_folder_today, exist_ok=True)

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Get list of files in today's processing folder
        files = os.listdir(processing_folder_today)

        # Iterate through each file
        for file_name in files:
            # Move file to completed folder
            source_path = os.path.join(processing_folder_today, file_name)
            destination_path = os.path.join(completed_folder_today, file_name)
            shutil.move(source_path, destination_path)

            # Update SourceFile table
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute('''UPDATE SourceFile 
                              SET status=?, updated_datetime=? 
                              WHERE source_file=?''', 
                           ('Ready', now, file_name + ".wav"))
            conn.commit()

            # Path to chunks folder for the current file
            file_folder = os.path.join(completed_folder_today, file_name)
            chunks_folder = os.path.join(file_folder, "Chunks")
            
            # Check if chunks folder exists
            if os.path.exists(chunks_folder):
                # Get list of files in the chunks folder
                chunks_files = os.listdir(chunks_folder)
                
                # Iterate through each chunk file
                for chunk_file in chunks_files:
                    try:
                        # Update ProcessedFile table for chunk file
                        cursor.execute('''INSERT INTO ProcessedFile 
                                        (local_file, source_file, status, chunk_created_datetime, updated_datetime) 
                                        VALUES (?, ?, ?, ?, ?)''', 
                                    (chunk_file, file_name, 'Completed', now, now))
                        conn.commit()
                    except Exception as e:
                        print(f"Error updating ProcessedFile table for chunk {chunk_file}: {str(e)}")

        # Close the database connection
        conn.close()
        print("All files processed successfully.")
    except Exception as e:
        print(f"Error moving files to completed folder: {str(e)}")

# Call the function to move files to completed folder and update database
move_files_to_completed()


All files processed successfully.


In [36]:
import os
import shutil
import sqlite3
from datetime import datetime
import logging

def move_files_to_completed():
    try:
        # Constants
        processing_folder = "processing/"
        completed_folder = "completed/"

        # Get current date
        today_date = datetime.now().strftime("%y%m%d")

        # Path to today's processing folder
        processing_folder_today = os.path.join(processing_folder, today_date)
        if not os.path.exists(processing_folder_today):
            logging.info(f"No files found for today's date {today_date} in the processing folder.")
            return
        
        # Path to today's completed folder
        completed_folder_today = os.path.join(completed_folder, today_date)
        os.makedirs(completed_folder_today, exist_ok=True)

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Get list of files in today's processing folder
        files = os.listdir(processing_folder_today)

        # Iterate through each file
        for file_name in files:
            # Move file to completed folder
            source_path = os.path.join(processing_folder_today, file_name)
            destination_path = os.path.join(completed_folder_today, file_name)
            logging.info(f"Moving file {file_name} to completed folder...")

            # Check if source file exists
            if os.path.exists(source_path):
                try:
                    # Move file
                    shutil.move(source_path, destination_path)
                    logging.info(f"File {file_name} moved to completed folder.")

                    # Update SourceFile table
                    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    cursor.execute('''UPDATE SourceFile 
                                      SET status=?, updated_datetime=? 
                                      WHERE source_file=?''', 
                                   ('Ready', now, file_name + ".wav"))
                    conn.commit()

                    # Path to chunks folder for the current file
                    file_folder = os.path.join(completed_folder_today, file_name)
                    chunks_folder = os.path.join(file_folder, "Chunks")
                    
                    # Check if chunks folder exists
                    if os.path.exists(chunks_folder):
                        # Get list of files in the chunks folder
                        chunks_files = os.listdir(chunks_folder)
                        
                        # Iterate through each chunk file
                        for chunk_file in chunks_files:
                            try:
                                # Update status and updated_datetime columns in ProcessedFile table for chunk file
                                cursor.execute('''UPDATE ProcessedFile 
                                                SET status=?, updated_datetime=? 
                                                WHERE local_file=?''', 
                                                ('Completed', now, chunk_file))
                                conn.commit()
                            except Exception as e:
                                logging.error(f"Error updating ProcessedFile table for chunk {chunk_file}: {str(e)}")
                except Exception as e:
                    logging.error(f"Error moving file {file_name}: {str(e)}")
            else:
                logging.error(f"Source file {source_path} does not exist.")

        # Close the database connection
        conn.close()
        logging.info("All files processed successfully.")
    except Exception as e:
        logging.error(f"Error moving files to completed folder: {str(e)}")

# Configure logging
log_file = 'logs/file_convert.log'
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Call the function to move files to completed folder and update database
move_files_to_completed()
